In [22]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
import spacy 
from tqdm import tqdm
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_csv("train.csv")
df.head()



,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
del df['id']

In [4]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
# ref: https://stackoverflow.com/questions/51994254/removing-url-from-a-column-in-pandas-dataframe

def cleaning(df):
    df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii')) # remove emojies https://stackoverflow.com/questions/57514169/how-can-i-remove-emojis-from-a-dataframe
    df['comment_text'] = df['comment_text'].apply(lambda x: re.split('http(s)?:\/\/.*', str(x))[0])  # removing urls from text
    df['comment_text'] = df['comment_text'].str.replace(r'[\t\n]', ' ')
    df['comment_text'] = df['comment_text'].str.replace(r'[^a-zA-Z0-9 \']', '') # removed all characters except alphanumeric data and whitespace
    df['comment_text'] = df['comment_text'].str.strip()
    return(df)


In [6]:
train = cleaning(df)
train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation Why the edits made under my userna...,0,0,0,0,0,0
1,D'aww He matches this background colour I'm se...,0,0,0,0,0,0
2,Hey man I'm really not trying to edit war It's...,0,0,0,0,0,0
3,More I can't make any real suggestions on impr...,0,0,0,0,0,0
4,You sir are my hero Any chance you remember wh...,0,0,0,0,0,0


In [7]:
print('Checking Null')
print(df.isnull().any())

Checking Null
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool


In [103]:
def toxcity(df):
    df['is_toxic'] = -1
    for i in range(len(df)):
        df['is_toxic'][i] = (1 if (df.toxic[i] or df.obscene[i] or df.severe_toxic[i] or df.threat[i] or df.insult[i] or df.identity_hate[i]) else 0)
    df.drop(df.iloc[:, 1:7], inplace=True, axis=1)
    return(df)

    

In [ ]:
train1 = toxcity(train)
train1.head()

In [3]:
train = pd.read_csv("new_train_data.csv")

In [4]:
train.head()

,comment_text,is_toxic
0,Explanation Why the edits made under my userna...,0
1,Daww He matches this background colour Im seem...,0
2,Hey man Im really not trying to edit war Its j...,0
3,More I cant make any real suggestions on impro...,0
4,You sir are my hero Any chance you remember wh...,0


In [5]:
def contractions(df):

    df['comment_text'] = df['comment_text'].str.lower()

    df['comment_text'] = df['comment_text'].replace( { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he has",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has",
    "i'd": "I had",
    "i'd've": "I would have",
    "i'll": "I shall",
    "i'll've": "I shall have",
    "i'm": "I am",
    "im" : "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it shall",
    "it'll've": "it shall have",
    "it's": "it has",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had",
    "she'd've": "she would have",
    "she'll": "she shall",
    "she'll've": "she shall have",
    "she's": "she has",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that has",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there has",
    "they'd": "they had",
    "they'd've": "they would have",
    "they'll": "they shall",
    "they'll've": "they shall have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall",
    "what'll've": "what shall have",
    "what're": "what are",
    "what's": "what has",
    "what've": "what have",
    "when's": "when has",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has",
    "where've": "where have",
    "who'll": "who shall",
    "who'll've": "who shall have",
    "who's": "who has",
    "who've": "who have",
    "why's": "why has",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you shall",
    "you'll've": "you shall have",
    "you're": "you are",
    "you've": "you have"
    })
    return(df)



In [6]:
train1 = contractions(train)
train1.to_csv('new_train_data2.csv',index=False)
train1.head()

,comment_text,is_toxic
0,explanation why the edits made under my userna...,0
1,daww he matches this background colour im seem...,0
2,hey man im really not trying to edit war its j...,0
3,more i cant make any real suggestions on impro...,0
4,you sir are my hero any chance you remember wh...,0


In [7]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [165]:
word_Lemmatized = WordNetLemmatizer()
for index, comment in enumerate(train1['comment_text']):

    temp_data = []
    for word in str(comment).split():
        if word not in stopwords and word.isalpha():
            temp_word = word_Lemmatized.lemmatize(word)
            temp_data.append(temp_word)
    train1.loc[index,'comment_text'] = str(" ".join(temp_data))

In [ ]:
train1.to_csv('new_train_data3.csv',index=False)
train1.head()

In [3]:
train = pd.read_csv("new_train_data3.csv")
train = train[train['comment_text'].notna()]
train.isnull().values.any()

False

In [4]:
train.head()

,comment_text,is_toxic
0,explanation edits made username hardcore metal...,0
1,daww match background colour im seemingly stuc...,0
2,hey man im really not trying edit war guy cons...,0
3,cant make real suggestion improvement wondered...,0
4,sir hero chance remember page thats,0


In [5]:
train2 = pd.DataFrame(train['comment_text'], columns = ['comment_text']) 
y_train1 = pd.DataFrame(train['is_toxic'], columns = ['is_toxic']) 

## Splitting of Dataset

In [6]:
X_train, X_test, y_train, y_test = train_test_split(train2,y_train1, test_size=0.20, stratify = y_train1)
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.20, stratify=y_train)

In [7]:
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
X_train['comment_text'] = X_train['comment_text'].apply(lambda x: str(x))
X_train['comment_text'] = X_train['comment_text'].apply(lambda x: str(x))
X_cv['comment_text'] = X_cv['comment_text'].apply(lambda x: str(x))
X_cv['comment_text'] = X_cv['comment_text'].apply(lambda x: str(x))
X_test['comment_text'] = X_test['comment_text'].apply(lambda x: str(x))
X_test['comment_text'] = X_test['comment_text'].apply(lambda x: str(x))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


## BAG OF WORDS


In [8]:
# We are considering only the words which appeared in at least 10 documents(rows or projects).

from sklearn.feature_extraction.text import CountVectorizer
bow_vec = CountVectorizer(ngram_range=(1,2),min_df=10,max_features=5000)
bow_vec.fit(X_train['comment_text'].values.astype('U')) 


# we use the fitted CountVectorizer to convert the text to vector
X_train_text_bow = bow_vec.transform(X_train['comment_text'].values)
X_cv_text_bow = bow_vec.transform(X_cv['comment_text'].values)
X_test_text_bow = bow_vec.transform(X_test['comment_text'].values)


print("After vectorizations")
print(X_train_text_bow.shape, y_train.shape)
print(X_cv_text_bow.shape, y_cv.shape)
print(X_test_text_bow.shape, y_test.shape)
print("="*100)

After vectorizations
(101948, 5000) (101948, 1)
(25487, 5000) (25487, 1)
(31859, 5000) (31859, 1)


In [9]:
print(X_train[:7])

                                             comment_text
61220                           read become familiar wpor
108361  john cena cock lover adam rose latest rose bud...
80492   warning one vandalism mtv roadies season youll...
139835  bailey animated segment based beetle bailey co...
141178  user page quoted referring reverse christianit...
37942   thanks experimenting page chuck thomas wikiped...
65109   serious calling someone fag worse several comm...


## TFIDF vectorizer

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(ngram_range=(1,2),min_df=10,max_features=5000)
tfidf_vec.fit(X_train['comment_text'].values.astype('U')) 

# we use the fitted CountVectorizer to convert the text to vector
X_train_text_tfidf = tfidf_vec.transform(X_train['comment_text'].values.astype('U'))
X_cv_text_tfidf = tfidf_vec.transform(X_cv['comment_text'].values.astype('U'))
X_test_text_tfidf = tfidf_vec.transform(X_test['comment_text'].values.astype('U'))


print("After vectorizations")
print(X_train_text_tfidf.shape, y_train.shape)
print(X_cv_text_tfidf.shape, y_cv.shape)
print(X_test_text_tfidf.shape, y_test.shape)
print("="*100)


After vectorizations
(101948, 5000) (101948, 1)
(25487, 5000) (25487, 1)
(31859, 5000) (31859, 1)


In [11]:
word2tfidf = dict(zip(tfidf_vec.get_feature_names(), tfidf_vec.idf_))

In [12]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

comment_text = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_train['comment_text'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    comment_text.append(mean_vec1)
print(len(comment_text))
print(len(comment_text[0]))
X_train['comment_text1'] = list(comment_text )

100%|█████████████████████████████████████████████████████████████████████████| 101948/101948 [15:43<00:00, 108.04it/s]


101948
96


In [14]:
print(X_train['comment_text1'][:10])

61220     [26.96457290649414, -13.232075214385986, -51.4...
108361    [555.3385216891766, -373.703430891037, -25.839...
80492     [61.11747074127197, -88.30073118209839, 9.4977...
139835    [22.60871982574463, -156.15423965454102, -59.0...
141178    [178.26434314250946, -159.68696177005768, -88....
37942     [247.19200491905212, -148.3308681845665, -79.8...
65109     [204.2484426498413, -321.8095157444477, -194.3...
106333    [622.4905781149864, -564.826981626451, -310.24...
33329     [64.08963990211487, -70.9236102104187, 5.47227...
142954    [-31.569822937250137, -81.60025405883789, -22....
Name: comment_text1, dtype: object


In [15]:
XY = pd.DataFrame(columns = ['comment_text'])
XY['comment_text'] = list(comment_text)
df_new = pd.DataFrame(XY.comment_text.values.tolist(), index= XY.index)

In [16]:
df_new.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,26.964573,-13.232075,-51.423710,-68.825533,7.040286,13.651844,82.660530,-19.930314,13.725100,12.310110,...,40.463550,39.369254,39.721000,10.145798,17.784253,5.819193,5.070313,7.670372,-3.465498,0.513311
1,555.338522,-373.703431,-25.839227,-371.843868,-217.778643,-63.570403,1238.976497,-123.737944,-134.315726,3.419208,...,239.193009,51.640220,444.625114,43.415027,-48.124402,-79.899778,-410.740567,-174.988344,-462.403376,64.217246
2,61.117471,-88.300731,9.497729,-63.071559,-33.923425,-26.887755,124.325799,-27.527881,-38.047474,21.823889,...,89.233208,7.157426,67.867073,-14.546132,36.244981,-14.973471,-17.900226,21.431245,-92.318309,64.195417
3,22.608720,-156.154240,-59.018239,-76.727845,-3.638053,70.106097,241.927475,-76.655878,-35.887948,30.423056,...,17.118950,19.842059,131.816375,-34.960539,18.312917,0.810375,17.523484,-104.066650,-111.987121,68.371448
4,178.264343,-159.686962,-88.821998,-75.602972,-47.375468,95.177323,555.907910,-42.609272,29.195191,78.070279,...,182.501729,-60.288521,280.266837,-109.263897,-21.995158,-35.778953,-186.152998,-16.846743,-264.263888,146.973720


In [17]:
# en_vectors_web_lg, which includes over 1 million unique vectors.


comment_text = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_cv['comment_text'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    comment_text.append(mean_vec1)
print(len(comment_text))
print(len(comment_text[0]))
X_cv['comment_text1'] = list(comment_text )

100%|████████████████████████████████████████████████████████████████████████████| 25487/25487 [04:07<00:00, 87.39it/s]


25487
96


In [18]:
XY1 = pd.DataFrame(columns = ['comment_text'])
XY1['comment_text'] = list(comment_text)
df_cv_new = pd.DataFrame(XY1.comment_text.values.tolist(), index= XY1.index)

In [19]:
df_cv_new.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,111.527193,-236.061052,-374.842996,-503.636997,-375.254875,269.588323,679.568324,-8.151155,-126.396486,77.130225,...,456.146406,-218.110089,521.805377,48.181597,204.058845,-44.903806,-132.128291,-30.224331,-297.908171,78.783722
1,28.850231,-17.723409,19.317061,-9.612974,-30.565720,-9.520192,62.664947,5.304018,-38.129993,-13.719576,...,24.982732,-20.363069,27.695212,-7.462664,1.174664,-22.016214,12.403508,-0.341751,-10.040838,7.610307
2,267.287617,-191.523173,-84.474854,-268.345981,-106.770810,173.361692,465.595013,-48.109782,-205.437472,101.731941,...,124.606398,-146.703100,448.078590,-8.622227,-15.359532,-3.092044,-163.449561,-66.810140,-260.055141,134.020925
3,197.614381,-307.966216,-227.240266,-242.549247,-226.088004,112.266209,588.072838,28.854511,-14.719902,60.307329,...,258.885050,-66.389294,317.462677,54.062080,-35.141008,-68.953396,-198.937328,-33.224415,-259.942523,151.292140
4,-14.008488,25.694328,-71.258643,-75.303047,-22.307944,29.044970,96.748967,-33.850050,20.221398,-31.068036,...,62.306706,4.958614,20.385313,7.188056,-3.177866,-10.179092,6.006121,-14.496660,41.927819,-15.288059


In [20]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

comment_text = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_test['comment_text'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    comment_text.append(mean_vec1)
print(len(comment_text))
print(len(comment_text[0]))
X_test['comment_text1'] = list(comment_text )

100%|███████████████████████████████████████████████████████████████████████████| 31859/31859 [05:05<00:00, 104.20it/s]


31859
96


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
XY2 = pd.DataFrame(columns = ['comment_text'])
XY2['comment_text'] = list(comment_text)
df_test_new = pd.DataFrame(XY2.comment_text.values.tolist(), index= XY2.index)
df_test_new.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,-0.037096,27.603437,-22.416633,-73.819184,-36.534962,2.301421,-0.058319,0.483829,2.758542,24.481396,...,25.232072,-50.422961,-7.945901,10.632874,9.615028,26.038742,1.862943,-1.025613,14.028592,6.674342
1,20.661575,-137.725263,-32.087172,-80.298715,-55.547397,52.907312,314.244802,20.180854,-11.938745,-47.750197,...,117.231783,-37.818131,128.153005,-37.208856,-25.318992,-34.863788,-91.653792,-42.727061,-151.184678,108.775219
2,978.075718,-1126.239441,-671.041108,-1338.454208,-1092.220165,826.083679,2517.418896,-277.465775,-350.793687,683.392566,...,795.429192,-609.646322,1572.653834,-176.049227,19.518315,188.280949,-1119.575568,-351.484781,-1004.934110,358.543629
3,145.765818,-157.261655,-128.700238,-166.085837,-79.642886,13.356430,338.769338,-127.840959,-51.800542,21.576185,...,35.291060,-53.339952,210.401578,31.154771,-13.385899,-28.461925,-57.812566,-22.055548,-136.940964,57.731280
4,99.220429,-65.364222,-52.194325,-93.846159,-68.897620,-5.850902,112.369323,-48.667926,5.731299,75.487959,...,99.346541,-4.125181,106.142587,-20.689805,-5.750309,-23.012619,-44.905130,-6.539322,-63.819631,59.101050
